In [1]:
load("../Shared/orbits.sage") # Group orbits
load("../Shared/linalg.sage") # Auxiliary linear algebra
load("../Shared/cyclic_covers.sage") # Cyclic covers of function fields
load("../Shared/weil_poly_utils.sage") # Utility functions for Weil polynomials
import itertools, pandas, time
from collections import defaultdict

In [2]:
# 5 elliptic curves over F_2 up to isom in Weierstrass form
ec_coeff = [[0, 0, 1, 1, 1], [1, 0, 1, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 1],[0, 0, 1, 1, 0]]
e_lst = []
for coeff in ec_coeff:
    e_lst.append(EllipticCurve(GF(2),coeff))
# p_rank = [0,1,0,1,0] (ordinary = 1, supersingular = 0)
# endomorphism rings are all ZZ
    
e_pc = [] #point counts of EC, E(F_2) = [1,2,3,4,5]
for e in e_lst:
    e_pc.append(tuple(e.count_points(6)))
    
R.<T> = PolynomialRing(QQ) 
e_wp = [T^2 + (-i + 3)*T + 2 for i in range(1,6)]

In [3]:
load("../Shared/pointcounts_p_rank.sage") # data = ...

In [4]:
# point count check:
# C -> E is of degree 2, so 2*#E(F_2^i) >= #C(F_2^i) 
# every F_2^i rational point of E lifts to either F_2^i rational ramification point or two F_2^2i rational points
# of C, so #C(F_2^2i) >= # E(F_2^i) - t

d = {}
for ecpc in e_pc:
    for cpc in data:
        if all(2*ecpc[i] >= cpc[0][i] for i in range(6)):
            try:
                d[ecpc].append(cpc)
            except:
                d[ecpc] = [cpc]

In [5]:
# resultant criterion: Cor 9.3 of rel class number I
# Deuring-Shafarevich formula: gamma_C -1 = 2*(gamma_E -1) + t   (Doesn't reduce the list)

newd = {}
for i in range(5):
    epc = list(d.keys())[i]
    ewp = e_wp[i]
    for cpc in d[epc]:
        u = weil_poly_from_point_count(list(cpc[0]),6)
        if gcd(ewp,u) > 1:
            g = u//ewp
            h1 = ewp.trace_polynomial()[0] #real Weil polys
            h2 = g.trace_polynomial()[0]
            if h1.resultant(h2) %2 == 0: #gcd(d=2,res(h1,h2)) = 2
                if i % 2 == 0 and (-1 < cpc[1] < 5):
                    try:
                        newd[epc].append(cpc)
                    except:
                        newd[epc] = [cpc]
                elif i % 2 == 1 and (0 < cpc[1] < 7):
                    try:
                        newd[epc].append(cpc)
                    except:
                        newd[epc] = [cpc]

In [8]:
d.clear()

In [7]:
[(len(newd[c]),c) for c in newd.keys()]

[(535, (1, 5, 13, 25, 41, 65)),
 (1142, (2, 8, 14, 16, 22, 56)),
 (1719, (3, 9, 9, 9, 33, 81)),
 (781, (4, 8, 4, 16, 44, 56)),
 (1007, (5, 5, 5, 25, 25, 65))]

In [16]:
#[points counts up to F_2^6, p-rank]
for i in range(5):
    file = open("data/" + str(i+1) + ".txt",'w')
    lst = newd[list(newd.keys())[i]]
    for pt in lst:
        tmp = pt[0] + [pt[1]]
        file.write(str(list(tmp)) + "\n")
    file.close()